In [31]:
import pandas_datareader.data as web
import pandas as pd

treasury_yield = web.DataReader('TB3MS', 'fred', pd.to_datetime("1999-01-01"), pd.to_datetime("2024-06-30"))
# yields are in percent per annum
treasury_yield = treasury_yield.divide(100) 
treasury_yield_per_month = treasury_yield.resample("ME").last().divide(12)
treasury_yield_per_month = treasury_yield_per_month.rename({"TB3MS": "rf"}, axis = 1)
treasury_yield_per_month.head()

,rf
DATE,
1999-01-31,0.003617
1999-02-28,0.003700
1999-03-31,0.003700
1999-04-30,0.003575
1999-05-31,0.003750


In [32]:
from utils import get_ff_factors

ff = get_ff_factors()
ff.reset_index(inplace = True)
ff.loc[:, "date"] = pd.to_datetime(ff.date)
ff.head()

,date,Mkt-RF,SMB,HML,RF
0,1926-07-01 00:00:00,0.0010,-0.0025,-0.0027,0.00009
1,1926-07-02 00:00:00,0.0045,-0.0033,-0.0006,0.00009
2,1926-07-06 00:00:00,0.0017,0.0030,-0.0039,0.00009
3,1926-07-07 00:00:00,0.0009,-0.0058,0.0002,0.00009
4,1926-07-08 00:00:00,0.0021,-0.0038,0.0019,0.00009


In [34]:
import requests

url = f'https://eodhd.com/api/eod/A.US?api_token=64d34edad4d0c3.90832312&fmt=json'
data = requests.get(url).json()

df_tmp = pd.DataFrame(data)
df_tmp.loc[:, "ticker"] = "A"
df_tmp.loc[:, "date"] = pd.to_datetime(df_tmp.date)
df_tmp

,date,open,high,low,close,adjusted_close,volume,ticker
0,1999-11-18 00:00:00,45.5000,50.0000,40.0000,44.0000,26.6514,62546380,A
1,1999-11-19 00:00:00,42.9375,43.0000,39.8125,40.3749,24.4556,15234146,A
2,1999-11-22 00:00:00,41.3124,44.0000,40.0625,44.0000,26.6514,6577870,A
3,1999-11-23 00:00:00,42.5000,43.6250,40.0000,40.0000,24.2285,5975611,A
4,1999-11-24 00:00:00,40.1250,41.9375,40.0000,41.0625,24.8721,4843231,A
...,...,...,...,...,...,...,...,...
6172,2024-06-03 00:00:00,130.0000,132.5800,130.0000,131.4000,131.4000,3114800,A
6173,2024-06-04 00:00:00,131.2400,132.2500,130.2800,130.8500,130.8500,2893700,A
6174,2024-06-05 00:00:00,130.5400,134.2000,130.0100,133.5600,133.5600,2707500,A
6175,2024-06-06 00:00:00,132.4600,134.3500,132.2800,132.8200,132.8200,2269600,A


In [35]:
from utils import rolling_ff_regression
import numpy as np

# determine momentum and target variable (monthly return for t+1)
df_tmp_monthly = df_tmp.resample("ME", on = "date").last()
df_tmp_monthly.loc[:, "mom1m"] = df_tmp_monthly.adjusted_close.pct_change()
df_tmp_monthly.loc[:, "mom6m"] = df_tmp_monthly.loc[:, "adjusted_close"].pct_change(6)
df_tmp_monthly.loc[:, "ch6mom"] = df_tmp_monthly.loc[:, "mom6m"].diff()
df_tmp_monthly.loc[:, "mom12m"] = df_tmp_monthly.loc[:, "adjusted_close"].pct_change(12)
df_tmp_monthly = df_tmp_monthly.merge(treasury_yield_per_month, left_index=True, right_index=True)
df_tmp_monthly.loc[:, "excess_rt"] = df_tmp_monthly.mom1m - df_tmp_monthly.rf
df_tmp_monthly.loc[:, "target"] = df_tmp_monthly.loc[:, "excess_rt"].shift(-1)

# determine rolling FF3 betas and idiosyncratic risk
df_tmp.loc[:, "rt"] = df_tmp.adjusted_close.pct_change()
df_tmp = df_tmp.merge(ff, on = "date")
df_tmp.loc[:, "excess_rt"] = df_tmp.rt - df_tmp.RF
rolling_betas = rolling_ff_regression(df_tmp, df_tmp.date)

# determine dollar volume
df_dolvol = df_tmp.loc[:, ['date', 'close', 'volume']]
df_dolvol = df_dolvol.assign(dollar_vol = lambda x: np.log(x["close"]*x["volume"]))
df_dolvol = df_dolvol.resample("ME", on = "date").last().drop(["close", "volume"], axis = 1)

# merge all and output
df_out = rolling_betas.merge(df_dolvol, left_index=True, right_index=True).merge(df_tmp_monthly.loc[:, ["mom1m", "mom6m", "ch6mom", "mom12m", "target"]], left_index = True, right_index = True)
df_out

/Users/ralf/Library/Mobile Documents/com~apple~CloudDocs/Kurse/SCDP/.venv/lib/python3.11/site-packages/pandas/core/groupby/grouper.py:381: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  ax = Index(obj[key], name=key)
/Users/ralf/Library/Mobile Documents/com~apple~CloudDocs/Kurse/SCDP/.venv/lib/python3.11/site-packages/pandas/core/groupby/grouper.py:381: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  ax = Index(obj[key], name=key)
/Users/ralf/Library/Mobile Documents/com~apple~CloudDocs/Kurse/SCDP/.venv/lib/python3.11/site-packages/pandas/core/indexes/base.py:7630: FutureWarning: Dtype inference on a pandas object (Series, Index, Extensi

,beta_m,retvol,dollar_vol,mom1m,mom6m,ch6mom,mom12m,target
1999-11-30,NaN,NaN,19.018578,NaN,NaN,NaN,NaN,0.828261
1999-12-31,NaN,NaN,18.821506,0.832594,NaN,NaN,NaN,-0.148331
2000-01-31,NaN,NaN,18.047600,-0.143897,NaN,NaN,NaN,0.564783
2000-02-29,NaN,NaN,18.415842,0.569408,NaN,NaN,NaN,-0.003539
2000-03-31,NaN,NaN,19.777098,0.001203,NaN,NaN,NaN,-0.152554
...,...,...,...,...,...,...,...,...
2023-10-31,0.956860,0.013906,19.250792,-0.073704,-0.233729,-0.044890,-0.247559,0.231945
2023-11-30,0.836801,0.014667,19.559163,0.236336,0.109209,0.342938,-0.169551,0.085345
2024-01-31,0.883000,0.014354,18.780396,-0.064231,0.072373,-0.088092,-0.138313,0.051436
2024-02-29,0.792345,0.014785,19.702282,0.055803,0.138761,0.066388,-0.025442,0.054967
